# Prompting Engineering

In [ ]:
# 주피터 환경에서 터미널이 아닌 코드로도 pip install 가능
%pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 환경설정
import os
from dotenv import load_dotenv

load_dotenv() # 현재 경로의 .env 파일을 읽어 시스템 환경변수로 등록

OPENAI_API_KEY = os.getenv('OPEN_API_KEY') # .env 파일안의 변수명과 일치하게 입력

## Chat Completion

In [6]:
from openai import OpenAI

# client = server
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role" : "system",     # 시스템에 기본 행동을 설정
            "content" : [
                {
                    "type" : "text",
                    "text" : "친절한 말투 사용하지말고 마치 부하직원을 하대하는 상사야"
                }
            ]
        },
        {
            "role" : "user",    # 유저 질문
            "content" : [
                {
                    "type" : "text",
                    "text" : "밖에 추운데 점심 추천좀 해줘. 10000원 안으로"
                }
            ]
        }
    ],
    response_format={"type" : "text"},    # 응답 형식
    temperature=1.3,     # 응답 창의성(기본값 1) 0 ~ 2
    max_tokens=2048,     # 응답 최대 토큰수
    top_p=1             # 사용할 상위 누적확률
)

print(response.choices[0].message.content)

밖이 춥다고? 그럼 뜨거운 국물부터 먹는 게 좋겠지. 김치찌개 집 찾든가, 아니면 곰탕 같은 거 어때? 10,000원 안에 맞출 수 있을 거야.


### 패턴1 : 페르소나 & Few-Shot(기사 제목 교정)

In [15]:
from openai import OpenAI

def correct_title(query, temperature=0.3):

    # client = server
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction = """
    넌 신문사 베테랑 편집장이야. 기사들이 송고한 제목을 교정해줘.

    ## 지시사항 ##
    - 기사의 제목이 명확하고 주제와 잘 맞도록 수정
    - 비속어, 은어 등은 제거하고 의미가 유지되도록 제목 교정
    - 간결하고 임택트 있는 표현 사용
    - 형식이 제목이랑 맞지 않을 경우 제목형태로 교정

    ## 출력 예시 ##
    - 원래 제목 : [기사의 원래 제목]
    - 교정 제목 : 
        [교정된 제목1]
        [교정된 제목2]

    ## 예시 ##
    - 원래 제목 : "어제 서울에서 큰불이 나서 수백명이 대피
    - 교정 제목 : 
        "서울 대형화재, 수백명 대피"
        "서울 대형 화재 발생, 수백명 대피, 아수라장"
    """

    user_message = f"""
    다음 제목을 교정해.
    제목 : {query}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role" : "system",     # 시스템에 기본 행동을 설정
                "content" : [
                    {
                        "type" : "text",
                        "text" : system_instruction     # 미리 정해놓은 페르소나를 전달하는 메소드
                    }
                ]
            },
            {
                "role" : "user",    # 유저 질문
                "content" : [
                    {
                        "type" : "text",
                        "text" : user_message   # query 파라미터로 전달받은 메세지를 전달하는 메소드
                    }
                ]
            }
        ],
        response_format={"type" : "text"},    # 응답 형식
        temperature=temperature,     # 파라미터로 전달하는 temperature
        max_tokens=2048,     # 응답 최대 토큰수
        top_p=1             # 사용할 상위 누적확률
    )

    return response.choices[0].message.content

print(correct_title('다시돌아온 테이의 개꿀잼 라디오 방송 ㄱㄱ'))
print(correct_title('아이돌 연애에 왜이렇게 급발진하고 욕하고 난리인거냐'))




    - 원래 제목 : 다시돌아온 테이의 개꿀잼 라디오 방송 ㄱㄱ
    - 교정 제목 : 
        "테이의 라디오 방송, 다시 시작"
        "테이의 라디오 복귀, 기대감 상승"

    - 원래 제목 : 아이돌 연애에 왜이렇게 급발진하고 욕하고 난리인거냐
    - 교정 제목 : 
        "아이돌 연애, 대중 반응 과열 논란"
        "아이돌 연애, 대중의 과도한 반응 문제"


### 구조화된 출력(Structured Ouptput) - 영단어장 생성

In [31]:
from openai import OpenAI
import json # API로부터 받은 JSON형식의 텍스트 응답을 파이썬 객체로 변환하는데 사용

def extract_eng_words(query, temperature=0.3):

    # client = server
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction = """
    넌 영어 팝송을 이용해 흥미롭고 이해하기 쉬운 방식으로 영어를 가르치는 선생님이야.

    ## 처리 단계 ##
    1. 주어진 질문이 팝송 제목인지 팝송 가사인지 먼저 구분
    2. 팝송 가사라면 주어진 가사에서 가장 많이 사용되는 단어 5개 추출
    3. 팝송 제목이라면 주어진 제목의 노래의 가사를 확인하고 가장 많이 사용되는 단어 5개 추출
    4. 각 단어의 의미를 한글로 알려줘
    5. 단아별로 유사한 단어도 함께 소개

    ## 출력 형식 ##
    출력 형식은 json 형식
    - 최상위 키는 "json_list" 이어야 함
    - "json_list"의 값은 단어별 json 객체들이 담긴 리스트이어야 함

    ## 출력 예시 ##
    {
        "json_list" : [
            {
                "제목" : "just two of us"
                "단어" : "two"
                "의미" : "둘, 숫자2"
                "유사어" : [
                    {
                        "유사단어1" : "second"
                    },
                    {
                        "유사단어2" : "..."
                    }
                ]
            }
        ]
    }
    """

    user_message = f"""
    노래 가사 or 노래 제목 : {query}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role" : "system",
                "content" : [
                    {
                        "type" : "text",
                        "text" : system_instruction
                    }
                ]
            },
            {
                "role" : "user",
                "content" : [
                    {
                        "type" : "text",
                        "text" : user_message   # query 파라미터로 전달받은 메세지를 전달하는 메소드
                    }
                ]
            }
        ],
        response_format={"type" : "json_object"},    # json 형식으로 응답을 받음("json_object")
        temperature=temperature,
        max_tokens=2048,
        top_p=1
    )


    return json.loads(response.choices[0].message.content)  # 응답 json문자열을 파이썬 객체로 변환 (json.loads())

lyrics = "just two of us"
result = extract_eng_words(lyrics,temperature=1)
words = result["json_list"]

if words:
    print(f"### 노래 제목: {words[0]['제목']} ###")

# 파이썬 객체로 변환되었기 때문에 for 반목문을 통해 각 요소를 순회하고 딕셔너리의 키를 이용해 원하는 데이터에 직접 접근 가능

for word in words :
    print(f"단어 : {word["단어"]}")
    print(f"의미 : {word["의미"]}")
    print(f"유사어 : {word["유사어"]}")

### 노래 제목: just two of us ###
단어 : just
의미 : 단지, 그냥
유사어 : [{'유사단어1': 'only'}, {'유사단어2': 'merely'}]
단어 : two
의미 : 둘, 숫자2
유사어 : [{'유사단어1': 'second'}, {'유사단어2': 'pair'}]
단어 : of
의미 : ~의
유사어 : [{'유사단어1': 'from'}, {'유사단어2': 'about'}]
단어 : us
의미 : 우리(를)
유사어 : [{'유사단어1': 'we'}, {'유사단어2': 'ours'}]
단어 : we
의미 : 우리(가)
유사어 : [{'유사단어1': 'us'}, {'유사단어2': 'ourselves'}]


### 생각의 사슬(Chain-of-Thought) (냉장고 털이 마스터) - ReAct
* Reason + Act 기법으로 현재 상황에 대한 통찰이후, 다음 행동에 대한 작성을 유도하는 기법

In [ ]:
from openai import OpenAI
import json # API로부터 받은 JSON형식의 텍스트 응답을 파이썬 객체로 변환하는데 사용

def my_refregator(query, temperature=0.3):

    # client = server
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction = """
    너는 냉장고에 있는 재료들을 활용하여 창의적이고 실용적인 저녁식사 아이디어를 제안하는 요리 전문가야.
    네 역할은 사용자로부터 제공받은 재료목록을 분석하고, 이를 활용할 수 있는 요리 아이디어를 구상해서,
    조리방법을 단계별로 상세히 설명하는 것이야.

    # 출력 예시
    1. 상황분석
        - 현재 가진 재료는 [사용자가 입력한 재료]
        - 주재료인 닭고기와 다양한 채소들이 있다.
        - 이 재료들은 스튜, 볶음, 찜요리에 적합.

    2. 행동계획
        - 가장 쉽게 만들수 있는 요리로 닭볶음탕을 제안.
        - 각 요리에 필요한 재료와 조리도구를 확인.
        - 부족한 재료가 있다면, 대체가능한 옵션을 제시.
        - 조리 과정을 단계별로 상세히 설명.
        - 맛을 더 향상시킬수 있는 팁과 주의사항 또한 제공.

    3. 실행
        - 여기에 상세레시피를 단계별로 작성합니다.
    """

    user_message=f"""
    사용자의 냉장고에는 현재 {query}가 있습니다.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role" : "system",
                "content" : [
                    {
                        "type" : "text",
                        "text" : system_instruction
                    }
                ]
            },
            {
                "role" : "user",
                "content" : [
                    {
                        "type" : "text",
                        "text" : user_message
                    }
                ]
            }
        ],
        response_format={"type" : "text"},
        temperature=temperature,
        max_tokens=2048,
        top_p=1
    )


    return response.choices[0].message.content

print(my_refregator("닭가슴살, 계란, 김치, 파프리카"))




1. 상황분석
   - 현재 가진 재료는 닭가슴살, 계란, 김치, 파프리카입니다.
   - 닭가슴살은 단백질이 풍부하고 다양한 요리에 활용 가능.
   - 계란은 부드러운 식감을 더해주며, 김치는 발효된 맛으로 요리에 깊이를 더해줌.
   - 파프리카는 색감과 아삭한 식감을 제공.
   - 이 재료들은 볶음밥이나 김치찜에 적합.

2. 행동계획
   - 가장 쉽게 만들 수 있는 요리로 '김치 닭가슴살 볶음밥'을 제안.
   - 필요한 재료: 닭가슴살, 계란, 김치, 파프리카, 밥, 간장, 참기름, 소금, 후추.
   - 필요한 조리도구: 프라이팬, 칼, 도마, 주걱.
   - 만약 밥이 없다면, 즉석밥이나 다른 곡류로 대체 가능.
   - 조리 과정을 단계별로 상세히 설명.
   - 맛을 더 향상시킬 수 있는 팁과 주의사항 제공.

3. 실행
   - **재료 준비**: 닭가슴살을 한 입 크기로 썰고, 파프리카는 잘게 다집니다. 김치는 먹기 좋은 크기로 자릅니다.
   - **계란 준비**: 계란을 풀어 소금과 후추로 간을 한 뒤, 프라이팬에 스크램블 에그를 만들어 따로 둡니다.
   - **닭가슴살 조리**: 같은 프라이팬에 기름을 두르고 닭가슴살을 넣어 익힙니다. 소금과 후추로 간을 합니다.
   - **김치와 파프리카 추가**: 닭가슴살이 익으면 김치와 파프리카를 넣고 함께 볶습니다.
   - **밥 추가**: 밥을 넣고 간장과 참기름을 약간 넣어 잘 섞어줍니다.
   - **계란 추가**: 마지막으로 스크램블 에그를 넣고 전체적으로 잘 섞어줍니다.
   - **마무리**: 불을 끄고, 필요하면 추가 간을 합니다. 접시에 담아내면 완성입니다.

   - **팁**: 
     - 김치의 양은 취향에 따라 조절하세요. 더 매콤한 맛을 원하면 김치 국물을 약간 추가해도 좋습니다.
     - 참기름은 마지막에 넣어 향을 살려줍니다.
     - 파프리카 대신 다른 채소를 추가해도 좋습니다.


### 면접 질문 생성

In [33]:
from openai import OpenAI
import json # API로부터 받은 JSON형식의 텍스트 응답을 파이썬 객체로 변환하는데 사용

def job_interveiw(job_posting, temperature=0.3):

    # client = server
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction = """
    너는 풀스택과 AI 개발 분야의 전문가이면서, 해당 분야의 면접 전문가야.
    사용자가 제공한 구인공고에 근거해서 핵심적인 질문들로 지원자를 평가할 수 있어야 해.
    """

    user_message=f"""
    아래 채용공고의 직무에 대해 예상 면접 질문과 모범답안을 작성하시오.
    
    -- 가이드 --
    하드스킬과 소프트스킬 2개의 섹션으로 나눠 작성하시오.
    각 섹션별로 2개의 질문과 답변을 준비하시오.

    -- 출력형식 --
    # 1. 하드스킬
        - 질문1. 
        - 답변1.

        - 질문2.
        - 답변2.

    # 2. 소프트스킬
        - 질문1.
        - 답변1.

        - 질문2.
        - 답변2.

    -- 채용공고 --
    {job_posting}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role" : "system",
                "content" : [
                    {
                        "type" : "text",
                        "text" : system_instruction
                    }
                ]
            },
            {
                "role" : "user",
                "content" : [
                    {
                        "type" : "text",
                        "text" : user_message
                    }
                ]
            }
        ],
        response_format={"type" : "text"},
        temperature=temperature,
        max_tokens=2048,
        top_p=1
    )


    return response.choices[0].message.content

job_posting = """
부서소개
- 최고의 IT 기술력과 노하우를 바탕으로 끊임없이 연구 개발하여 고객이 만족할 수 있는 최적의 금융 솔루션을 제공하는 Professional Software Development Team!

핵심가치
- Innovation + Cooperation + Ownership + Concentration

부서의 미션
- 서비스 경쟁력 강화 및 시장 지배력 강화
- 프로젝트 관리 효율성 제고
- 데이터 통합 및 업무 프로세스 개선
- 내 외부 사용자 만족을 위한 최적의 솔루션 제공
- 급변하는 IT 트렌드에 대응하는 기술 기반 마련

부서에서 하는 일
- Client & Server 소프트웨어 개발 및 운영
- 데이터 수집, 가공 자동화 및 관리 시스템 개발
- IT 기술과 금융업무의 전문성을 바탕으로 대외 프로젝트 진행
- 차세대 기술 기반 확보 및 상품 적용 방안 수립
- 빅데이터 & 텍스트 마이닝을 통한 정보의 고급화 개발
- REST-ful API, TCP/IP 통신 데이터 서비스 개발

부서의 자랑거리, 우리 부서만의 문화
- Client/Server, 내/외부 프로그램 등 다양한 분야를 아우르는 시스템 개발 업무 수행
- 믿음과 단합으로 이루어진 수평적 조직 문화
- 자유로운 의견 개진과 공유를 통한 창의적 개발 활동

추가사항
- 여러 개발 언어를 활용하지만 C#을 주로 사용합니다.
- 응용 프로그램, 웹 등 다양한 분야에 개발에 관심을 가지고, 새로운 영역에 도전하고 학습하는 자세를 지향합니다.
담당업무
- 소프트웨어 개발 및 운영


자격요건
- 학사 이상

- 경력 : 3년 이하

- 컴퓨터 관련 전공 또는 부트캠프 등 실무 중심 개발 교육 이수하신 분

- .NET, MS-SQL 프로그래밍 능력 보유하신 분


우대사항
- 데이터 조회 성능 향상에 대한 관심과 기초 튜닝 경험 보유하신 분

- REST API, FTP/sFTP, TCP/IP, gRPC 등 다양한 통신 방식 개발 경험 보유하신 분  

- LLM API, 챗봇 개발 등 신기술 활용에 열려 있으신 분


공통 우대사항
- 주식시장과 기업공시에 대한 기본 지식을 갖추신분

- 빠른 면접진행 및 출근가능자 우대


기타사항
- 긍정 마인드, 책임감, 솔선수범


채용절차

*2차 면접 상황에따라 생략될 수 있음
"""

print(job_interveiw(job_posting, temperature=1))



# 1. 하드스킬
- 질문1. 본 공고는 주로 C#과 .NET 플랫폼을 활용한 개발 능력을 중요시합니다. C#과 .NET을 사용하여 진행한 프로젝트 경험을 하나 소개해 주시고, 그 과정에서 어렵게 느꼈던 부분과 이를 해결한 방법을 설명해 주세요.
- 답변1. 이전 회사에서 C#과 .NET을 활용하여 금융 관리 시스템을 개발한 경험이 있습니다. 해당 프로젝트의 가장 큰 도전은 대량의 데이터를 효율적으로 처리하고 빠르게 클라이언트에게 제공하는 것이었습니다. 문제를 해결하기 위해, Entity Framework의 성능을 최적화하는 연구를 진행했고, 최적화된 LINQ 쿼리를 설계하여 성능을 크게 향상시켰습니다. 또한 SQL Server에서 인덱스를 조정하여 데이터 조회 속도를 높였습니다.

- 질문2. REST API와 TCP/IP 통신 서비스 개발 경험이 강조되고 있습니다. 관련 기술을 활용하여 시스템을 설계했던 경험을 이야기해 주시고, 그 과정에서 확인된 베스트 프랙티스 중 하나를 설명해 주세요.
- 답변2. 이전에 RESTful API를 사용하여 고객 관리 시스템을 개발한 경험이 있습니다. 이 프로젝트에서 가장 중요하게 여겼던 것은 API의 보안성과 확장성이었습니다. 이를 위해 OAuth 2.0 인증 및 권한 부여를 사용하여 보안을 강화하고, 클라이언트가 필요로 하는 다양한 데이터 형식을 제공할 수 있도록 설계를 유연하게 했습니다. 또한, TCP/IP를 통해 실시간 데이터를 송수신하는 기능을 구현하여 시스템의 실시간성을 높였습니다.

# 2. 소프트스킬
- 질문1. 공고에 따르면 자유로운 의견 개진과 공유가 중요한 문화입니다. 이전 경험에서 팀원들과의 의견 차이를 극복하고, 협력하여 성공적으로 프로젝트를 완수한 경험이 있나요?
- 답변1. 예, 한 프로젝트에서 팀원과 데이터베이스 설계 방향에 대해 의견 차이가 있었습니다. 저는 수평적 조직 문화를 바탕으로 서로의 의견을 경청하고, 데이터 기반으로 논의를 이어갔습니다. 팀원들과의 회의를 통해 서로의 입장을 이해하고